In [3]:
import numpy as np

In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def create_label_array_from_logic(list_path):
    # 1. Định nghĩa lại selected_features y hệt như code gốc
    selected_features = ['B04B', 'B10B', 'B11B', 'B16B', 'IRB',
                         'CAPE', 'R850', 'TCWV', 'U850', 'I2B', 'TCLW', 'TCW']

    # Thêm 'y' vào danh sách bắt buộc (vì code gốc yêu cầu đủ 12 band + y)
    required_bands_list = sorted(list(set(selected_features + ['y'])))
    required_len = len(required_bands_list) # Thường là 13

    # 2. Đọc và gộp file CSV (Làm y hệt bước 1 code gốc)
    dfs = []
    print("Đang đọc CSV để lấy timeline chuẩn...")
    for p in list_path:
        # Chỉ cần đọc 2 cột này để check ngày, cho nhẹ
        df = pd.read_csv(p, usecols=['timestamp', 'variable'])
        df["variable"] = df["variable"].astype(str)

        # QUAN TRỌNG: Logic đổi tên biến nhãn của code gốc
        df.loc[df["variable"].isin(['2019', '2020']), "variable"] = 'y'
        dfs.append(df)

    df_all = pd.concat(dfs, ignore_index=True)

    # 3. Lọc Timestamp hợp lệ (Y hệt bước 2 & 4 code gốc)
    # Chỉ giữ lại các dòng thuộc các variable quan tâm
    df_check = df_all[df_all["variable"].isin(required_bands_list)]

    # Đếm số lượng variable unique trong mỗi timestamp
    ts_counts = df_check.groupby("timestamp")["variable"].nunique()

    # LOGIC CỐT LÕI: Chỉ lấy timestamp có đủ 13 variable (12 features + 1 y)
    valid_ts_index = ts_counts[ts_counts == required_len].index

    # Sort để khớp thứ tự với mảng x
    ts_valid = sorted(list(valid_ts_index))

    print(f"-> Đã tìm thấy {len(ts_valid)} timestamp khớp điều kiện (Kỳ vọng: 1223).")

    # 4. Tạo mảng l từ danh sách ts_valid này
    labels = []
    for t in ts_valid:
        ts = pd.to_datetime(t)
        if ts.month == 4:
            labels.append(0)
        elif ts.month == 10:
            labels.append(1)
        else:
            labels.append(0) # Xử lý nếu có tháng khác

    return np.array(labels)

# --- CÁCH SỬ DỤNG ---
# Thay list_path bằng đường dẫn thực tế đến các file csv của bạn
list_csv_files = [
    'csv_data/HIMA_hatinh.csv',
    'csv_data/ERA5_hatinh.csv',
    'csv_data/RADAR_hatinh.csv'
]

l = create_label_array_from_logic(list_csv_files)

print("\nKẾT QUẢ:")
print("Shape của l:", l.shape)
print("Các giá trị đầu:", l[:10])
print("Số lượng nhãn 0 (Tháng 4):", np.sum(l == 0))
print("Số lượng nhãn 1 (Tháng 10):", np.sum(l == 1))

Đang đọc CSV để lấy timeline chuẩn...
-> Đã tìm thấy 1223 timestamp khớp điều kiện (Kỳ vọng: 1223).

KẾT QUẢ:
Shape của l: (1223,)
Các giá trị đầu: [0 0 0 0 0 0 0 0 0 0]
Số lượng nhãn 0 (Tháng 4): 640
Số lượng nhãn 1 (Tháng 10): 583


In [17]:
print(l)

[0 0 0 ... 1 1 1]


In [18]:
import numpy as np

# Giả sử l là list bạn đã tạo xong từ bước trước
# Ví dụ: l = [1, 0, 0, 1, 1, 0, ...]

# 1. Chuyển list thành numpy array
l_array = np.array(l)

# Kiểm tra lại shape xem chuẩn chưa (phải là (1223,) hoặc tương ứng số lượng mẫu)
print("Shape của mảng nhãn:", l_array.shape)

# 2. Lưu xuống file .npy
# Bạn có thể đặt tên file là 'y_label.npy' hoặc tên gì tùy thích
output_filename = 'csv_data/y_t4_t10.npy'
np.save(output_filename, l_array)

print(f"Đã lưu thành công vào file: {output_filename}")

# --- (Phần test lại) ---
# Thử load lên lại xem ngon lành chưa
l_loaded = np.load(output_filename)
print("Load lại thử:", l_loaded[:10]) # In 10 cái đầu check chơi

Shape của mảng nhãn: (1223,)
Đã lưu thành công vào file: csv_data/y_t4_t10.npy
Load lại thử: [0 0 0 0 0 0 0 0 0 0]
